In [24]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras

In [23]:
print("hellO")

hellO


## Path defined

In [32]:
genuine_image_paths = r"C:\Users\Jeffrey\.vscode\projects\jupyter_projects\AxisBankAiChallenge-master\real/"
forged_image_paths = r"C:\Users\Jeffrey\.vscode\projects\jupyter_projects\AxisBankAiChallenge-master\forged/"

## Preprocessing the image

In [33]:
def rgbgrey(img):
    # Converts rgb to grayscale
    greyimg = np.zeros((img.shape[0], img.shape[1]))
    for row in range(len(img)):
        for col in range(len(img[row])):
            greyimg[row][col] = np.average(img[row][col])
    return greyimg

In [34]:
def greybin(img):
    # Converts grayscale to binary
    blur_radius = 0.8
    img = ndimage.gaussian_filter(img, blur_radius)  # to remove small components or noise
#     img = ndimage.binary_erosion(img).astype(img.dtype)
    thres = threshold_otsu(img)
    binimg = img > thres
    binimg = np.logical_not(binimg)
    return binimg

In [35]:
def preproc(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)
    if display:
        plt.imshow(img)
        plt.show()
    grey = rgbgrey(img) #rgb to grey
    if display:
        plt.imshow(grey, cmap = matplotlib.cm.Greys_r)
        plt.show()
    binimg = greybin(grey) #grey to binary
    if display:
        plt.imshow(binimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    r, c = np.where(binimg==1)
    # Now we will make a bounding box with the boundary as the position of pixels on extreme.
    # Thus we will get a cropped image with only the signature part.
    signimg = binimg[r.min(): r.max(), c.min(): c.max()]
    if display:
        plt.imshow(signimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    return signimg

## Feature Extraction


In [36]:
def Ratio(img):
    a = 0
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                a = a+1
    total = img.shape[0] * img.shape[1]
    return a/total

In [37]:
def Centroid(img):
    numOfWhites = 0
    a = np.array([0,0])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                b = np.array([row,col])
                a = np.add(a,b)
                numOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/numOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]

In [38]:
def EccentricitySolidity(img):
    r = regionprops(img.astype("int8"))
    return r[0].eccentricity, r[0].solidity

In [39]:
def SkewKurtosis(img):
    h,w = img.shape
    x = range(w)  # cols value
    y = range(h)  # rows value
    #calculate projections along the x and y axes
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    #centroid
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    #standard deviation
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    
    #skewness
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)

    #Kurtosis
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    # 3 is subtracted to calculate relative to the normal distribution
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)

In [40]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = preproc(path, display=display)
    ratio = Ratio(img)
    centroid = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewness, kurtosis = SkewKurtosis(img)
    retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    return retVal

In [41]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    temp = getFeatures(path, display=display)
    features = (temp[0], temp[1][0], temp[1][1], temp[2], temp[3], temp[4][0], temp[4][1], temp[5][0], temp[5][1])
    return features

## Saving the features

In [48]:
def makeCSV():
    if not(os.path.exists(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features')):
        os.mkdir(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features')
        print('New folder "Features" created')
    if not(os.path.exists(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features/Training')):
        os.mkdir(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-masterr\\Dataset\\Features/Training')
        print('New folder "Features/Training" created')
    if not(os.path.exists(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features/Testing')):
        os.mkdir(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features/Testing')
        print('New folder "Features/Testing" created')
    # genuine signatures path
    gpath = genuine_image_paths
    # forged signatures path
    fpath = forged_image_paths
    for person in range(1,13):
        per = ('00'+str(person))[-3:]
        print('Saving features for person id-',per)
        
        with open(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features\\Training/training_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Training set
            for i in range(0,3):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(0,3):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')
        
        with open(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features\\Testing/testing_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Testing set
            for i in range(3, 5):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(3,5):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')

In [49]:
makeCSV()

Saving features for person id- 001
Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


# TF Model 

In [53]:
def testing(path):
    feature = getCSVFeatures(path)
    if not(os.path.exists(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset/TestFeatures')):
        os.mkdir(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset/TestFeatures')
    with open(r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\TestFeatures/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, feature))+'\n')

In [67]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from time import time

n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\Features\\Training/training_'+train_person_id+'.csv'
testing(test_image_path)
test_path = r'C:\\Users\\Jeffrey\\.vscode\\projects\\jupyter_projects\\AxisBankAiChallenge-master\\Dataset\\TestFeatures/testcsv.csv'

def readCSV(train_path, test_path, type2=False):
    # Reading train data
    df = pd.read_csv(train_path, usecols=range(n_input))
    train_input = np.array(df.values)
    train_input = train_input.astype(np.float32, copy=False)  # Converting input to float_32
    df = pd.read_csv(train_path, usecols=(n_input,))
    temp = [elem[0] for elem in df.values]
    correct = np.array(temp)
    corr_train = keras.utils.to_categorical(correct, 2)  # Converting to one hot
    # Reading test data
    df = pd.read_csv(test_path, usecols=range(n_input))
    test_input = np.array(df.values)
    test_input = test_input.astype(np.float32, copy=False)
    if not type2:
        df = pd.read_csv(test_path, usecols=(n_input,))
        temp = [elem[0] for elem in df.values]
        correct = np.array(temp)
        corr_test = keras.utils.to_categorical(correct, 2)  # Converting to one hot
    if not type2:
        return train_input, corr_train, test_input, corr_test
    else:
        return train_input, corr_train, test_input

# Parameters
learning_rate = 0.001
training_epochs = 1000
display_step = 1

# Network Parameters
n_hidden_1 = 7  # 1st layer number of neurons
n_hidden_2 = 10  # 2nd layer number of neurons
n_hidden_3 = 30  # 3rd layer
n_classes = 2  # no. of classes (genuine or forged)

# Create model using Keras
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(n_hidden_1, activation='tanh', input_dim=n_input),
        tf.keras.layers.Dense(n_hidden_2, activation='tanh'),
        tf.keras.layers.Dense(n_hidden_3, activation='tanh'),
        tf.keras.layers.Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def evaluate(train_path, test_path, type2=False):
    if not type2:
        train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path)
    else:
        train_input, corr_train, test_input = readCSV(train_path, test_path, type2)

    model = build_model()

    # Training the model
    model.fit(train_input, corr_train, epochs=training_epochs, batch_size=32, verbose=0)

    # Evaluate accuracy
    train_accuracy = model.evaluate(train_input, corr_train, verbose=0)
    test_accuracy = model.evaluate(test_input, corr_test, verbose=0) if not type2 else None

    if not type2:
        return train_accuracy[1], test_accuracy[1]  # Accuracy is the second item in the returned tuple
    else:
        prediction = model.predict(test_input)
        predicted_class = np.argmax(prediction, axis=1)
        if predicted_class[0] == 1:
            print('Genuine Image')
            return True
        else:
            print('Forged Image')
            return False

def trainAndTest(rate=0.001, epochs=1700, neurons=7, display=False):
    start = time()

    # Parameters
    global learning_rate, training_epochs, n_hidden_1
    learning_rate = rate
    training_epochs = epochs

    # Network Parameters
    n_hidden_1 = neurons  # 1st layer number of neurons
    n_hidden_2 = 7  # 2nd layer number of neurons
    n_hidden_3 = 30  # 3rd layer

    train_avg, test_avg = 0, 0
    n = 10
    for i in range(1, n + 1):
        if display:
            print("Running for Person id", i)
        temp = ('0' + str(i))[-2:]
        train_score, test_score = evaluate(train_path.replace('01', temp), test_path.replace('01', temp))
        train_avg += train_score
        test_avg += test_score
    if display:
        print("Training average-", train_avg / n)
        print("Testing average-", test_avg / n)
        print("Time taken-", time() - start)
    return train_avg / n, test_avg / n, (time() - start) / n


evaluate(train_path, test_path, type2=True)


Enter person's id :  001
Enter path of signature image :  001001_001.png


FileNotFoundError: [Errno 2] No such file or directory: '001001_001.png'

In [60]:
import tensorflow as tf
print(tf.__version__)




2.18.0
